In [1]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:100])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
print('Length of dataset in characters: ', len(text))

Length of dataset in characters:  1115394


In [7]:
chars = sorted(list(set(text)))
vocab_size= len(chars)
print(''.join([i for i in chars]))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# Create a mapping from charactes to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # Take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # Take a list of integers, output a string

In [9]:
print(encode('Hello World'))
print(decode(encode('Hello World')))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [10]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [13]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, target is {target}")

When input is tensor([18]), target is 47
When input is tensor([18, 47]), target is 56
When input is tensor([18, 47, 56]), target is 57
When input is tensor([18, 47, 56, 57]), target is 58
When input is tensor([18, 47, 56, 57, 58]), target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [20]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [29]:
xb, yb = get_batch('train')
print('Input:')
print(xb.shape, xb)
print('Targets')
print(yb.shape, yb)

Input:
torch.Size([4, 8]) tensor([[57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46]])
Targets
torch.Size([4, 8]) tensor([[43, 60, 43, 52,  1, 63, 43, 39],
        [43, 42,  8,  0, 25, 63,  1, 45],
        [42,  5, 57,  1, 57, 39, 49, 43],
        [57, 58, 63,  6,  1, 58, 46, 47]])


In [30]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'When input is {context}, the target: {target}')


When input is tensor([57]), the target: 43
When input is tensor([57, 43]), the target: 60
When input is tensor([57, 43, 60]), the target: 43
When input is tensor([57, 43, 60, 43]), the target: 52
When input is tensor([57, 43, 60, 43, 52]), the target: 1
When input is tensor([57, 43, 60, 43, 52,  1]), the target: 63
When input is tensor([57, 43, 60, 43, 52,  1, 63]), the target: 43
When input is tensor([57, 43, 60, 43, 52,  1, 63, 43]), the target: 39
When input is tensor([60]), the target: 43
When input is tensor([60, 43]), the target: 42
When input is tensor([60, 43, 42]), the target: 8
When input is tensor([60, 43, 42,  8]), the target: 0
When input is tensor([60, 43, 42,  8,  0]), the target: 25
When input is tensor([60, 43, 42,  8,  0, 25]), the target: 63
When input is tensor([60, 43, 42,  8,  0, 25, 63]), the target: 1
When input is tensor([60, 43, 42,  8,  0, 25, 63,  1]), the target: 45
When input is tensor([56]), the target: 42
When input is tensor([56, 42]), the target: 5
Whe

In [31]:
import torch.nn as nn
from torch.nn import functional as F

In [34]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        # B = batch_size T = time C = channel
        logits = self.token_embedding_table(idx) # (B,T,C)
        # print(logits.shape)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):

        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            # Get the prediction
            logits, loss = self(idx)

            # Focus on only the last time step
            logits = logits[:, -1, :] # become (B, C)

            # Apply softmax to get probability
            probs = F.softmax(logits, dim=-1) # (B, C)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx



In [37]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [38]:
batch_size = 32
for steps in range(1000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.746829032897949


In [45]:
bath_size = 32
for steps in range(1000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.528167724609375


In [46]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx=idx, max_new_tokens=100)[0].tolist()))


NRDUL:
Wpr dsceomesgghe vSon aUheXforarkeli prJ bn yourirs n beat howhe hiusengpghQQghe fo$OVIUqRMis


# Self-attention

In [47]:
B, T, C = 4, 8, 2
x = torch.rand(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [48]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [68]:
wei  = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [69]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])